In [1]:
import torch
import torch_geometric

/home/icb/vnicolas.goedert/miniconda3/envs/RFAM/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rfam_dir = "../rfam/data/raw/processed/release-14.8"
rfams = ["RF00001","RF00174","RF00169","RF00050"]

from RNARepLearn.datasets import CombinedRfamDataset, SingleRfamDataset
#dataset = CombinedRfamDataset(rfam_dir, rfams, "Under300", 15, 300)
dataset = CombinedRfamDataset(rfam_dir, ["RF00001","RF00005"], "RF00001_RF00005")
#dataset = SingleRfamDataset(rfam_dir,"RF00001")

from RNARepLearn.utils import train_val_test_loaders

train_loader, val_loader, test_loader = train_val_test_loaders(dataset, 0.8, 0.05, 0.15)

Processing complete
7969
498
1495


Processing...
Done!


In [3]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("runs/TestRun")

In [7]:
from RNARepLearn.modules import LinearEmbedding, RPINetEncoder, AttentionDecoder
layers = []
layers.append(RPINetEncoder(4, 32, 3, 3))
layers.append(AttentionDecoder(32, 4))
model = torch.nn.Sequential(*layers)

In [8]:
from RNARepLearn.train import MaskedTraining, AutoEncoder
#training = MaskedTraining(model, 1, 15, writer)
training = AutoEncoder(model, 3, 15)

Creating new SummaryWriter


In [ ]:
training.run(train_loader, val_loader)

[Epoch    1/   3] [Batch    1/ 250] Loss:  1.59e+00 Nucleotide-Loss:  1.39e+00 Edge-Loss:  2.03e-01
[Epoch    1/   3] [Batch   11/ 250] Loss:  1.58e+00 Nucleotide-Loss:  1.38e+00 Edge-Loss:  2.01e-01
[Epoch    1/   3] [Batch   21/ 250] Loss:  1.56e+00 Nucleotide-Loss:  1.35e+00 Edge-Loss:  2.07e-01
[Epoch    1/   3] [Batch   31/ 250] Loss:  1.39e+00 Nucleotide-Loss:  1.21e+00 Edge-Loss:  1.79e-01
[Epoch    1/   3] [Batch   41/ 250] Loss:  1.18e+00 Nucleotide-Loss:  9.89e-01 Edge-Loss:  1.94e-01
[Epoch    1/   3] [Batch   51/ 250] Loss:  1.03e+00 Nucleotide-Loss:  8.52e-01 Edge-Loss:  1.83e-01
[Epoch    1/   3] [Batch   61/ 250] Loss:  9.95e-01 Nucleotide-Loss:  8.13e-01 Edge-Loss:  1.81e-01
[Epoch    1/   3] [Batch   71/ 250] Loss:  9.74e-01 Nucleotide-Loss:  7.85e-01 Edge-Loss:  1.89e-01
[Epoch    1/   3] [Batch   81/ 250] Loss:  9.44e-01 Nucleotide-Loss:  7.71e-01 Edge-Loss:  1.73e-01
[Epoch    1/   3] [Batch   91/ 250] Loss:  9.15e-01 Nucleotide-Loss:  7.61e-01 Edge-Loss:  1.54e-01


In [12]:
from RNARepLearn.modules import LinearEmbedding, RPINetEncoder, AttentionDecoderV2
layers = []
layers.append(RPINetEncoder(4, 32, 3, 3))
layers.append(AttentionDecoderV2(32, 4))
model = torch.nn.Sequential(*layers)
model.double()
model.to(training.device)

Sequential(
  (0): RPINetEncoder(
    (body): Sequential(
      (0): RPINetGNNLayer()
      (1): LSTM(32, 32)
      (2): RPINetGNNLayer()
      (3): LSTM(32, 32)
      (4): RPINetGNNLayer()
      (5): LSTM(32, 32)
    )
  )
  (1): AttentionDecoderV2(
    (nuc_projection): Linear(32, 4, bias=True)
    (attention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
    )
  )
)

In [13]:
batch = next(iter(test_loader))
true_x = torch.clone(batch.x)
true_edges = torch.clone(batch.edge_weight)
nucs, bpp = model(batch.to(training.device))

In [29]:
from RNARepLearn.utils import mask_batch, reconstruct_bpp
true_bpp = torch.tensor(reconstruct_bpp(batch.edge_index.cpu(), true_edges, (bpp.shape[0],bpp.shape[1])))

In [37]:
def BppReconstructionLoss(output, target):
    mask = target!=0
    loss = torch.nn.KLDivLoss(reduction="batchmean")

    print(output[mask])
    print(target[mask])
    print("-----------------")
    return(loss(output[mask], target[mask]))
BppReconstructionLoss(bpp.cpu().log(), true_bpp)

tensor([-8.0037, -8.0037, -8.0037,  ..., -8.0037, -8.0037, -8.0037],
       dtype=torch.float64, grad_fn=<IndexBackward0>)
tensor([9.0851e-03, 7.6014e-08, 7.5129e-10,  ..., 1.0507e-12, 8.2764e-13,
        9.9991e-01], dtype=torch.float64)
-----------------


tensor(0.2024, dtype=torch.float64, grad_fn=<DivBackward0>)